In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Multinomial Naive Bayes Classifier


We will implement a Naive Bayes Classifier in this first part of the lab.

## 1.1 Dataset

We will use an artificial dataset to evaluate the classifier will build. We will first upload this data. 

In [2]:
def bayes_dataset(split, prefix="bayes"):
    '''
    Arguments:
        split (str): "train" or "test"
    Returns:
        X (S x N): features of each object, X[i][j] = 0/1
        y (S): label of each object, y[i] = 0/1
    '''
    return (np.loadtxt(f"{prefix}_{split}_data.txt"), np.loadtxt(f"{prefix}_{split}_target.txt"))

In [3]:
data, target = bayes_dataset('train')
data.shape, target.shape, data[:5], target[:5]

((100, 20),
 (100,),
 array([[0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 1.],
        [0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
         0., 0., 1., 1.],
        [0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0.,
         0., 0., 0., 1.],
        [0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0.,
         0., 0., 0., 1.],
        [0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0.,
         0., 0., 1., 1.]]),
 array([1., 0., 0., 0., 0.]))

The training dataset $X\in\mathbb{R}^{N\times 20}$, each sample has 20 Boolean-valued features (i.e., take values that are either $0$ or $1$). There are 2 class labels, also $0$ or $1$. Our goal is to use the the features to make prediction on the labels.

We can think about the dataset in the following way. For example, we want to decide if we can Play golf or not based on a few weather conditions. Then
- feature 1 is "Rainy or not", 1 means Rainy, 0 means not Rainy. 
- feature 2 is "Cold or not", 1 means Cold, 0 means Hot.
- similar for all the other features. 
- the class label is "Play golf or not", 1 means Play Golf, 0 means not Play Golf.

## 1.2 Train

Consider 
- A training dataset $X\in\mathbb{R}^{N\times d}$.
- Training labels $Y\in\{0,1\}^{N}$. 
- We denote $X_{j}\in\mathbb{R}^N$ as the $j^{th}$ feature of $X$.

Recall from class, we can derive the classification rule as:

------
\begin{aligned}
\hat{Y} &=\underset{y}{\arg \max } P(Y=y \mid X) & \text{(Definition)}\\
&=\underset{y}{\arg \max } \frac{P(X \mid Y=y) P(Y=y)}{P(X)} & \text{(Bayes rule)}\\
&=\underset{y}{\arg \max } P(X \mid Y=y) P(Y=y) & \text{(Denominator does not depend on y)}\\
&=\underset{y}{\arg \max } P\left(X_1, \ldots, X_d \mid Y=y\right) P(Y=y) \\
&=\underset{y}{\arg \max }\left(\prod_{j=1}^d P\left(X_j \mid Y=y\right)\right) P(Y=y) & \text{(Conditional independence)} \\
&=\underset{y}{\arg \max }\left(\left(\sum_{j=1}^d \log P\left(X_j \mid Y=y\right)\right)+\log P(Y=y)\right) & \text{(Logarithmic operation)}
\end{aligned}

<!-- <img src="https://drive.google.com/uc?id=10Kz6CLP2HX1dQt95Wtbv3bUFnyB4n00q" width="800"/> -->

<br>

Therefore, when classifying a new data point $x = [x^{(1)}, ..., x^{(d)}]^\top$, we need to choose $y$ such that
$$\bigg[\log p(y) + \sum_{j=1}^d \log p(x^{(j)}|y) \bigg]$$ is the largest.

------

However, we first need to define the probabilistic models of the prior $p(y)$ and the class-conditional feature distributions $p(x^{(j)}|y)$ using the training data. In other words, we need to **train** the model.

### 1.2.1 Task 1: Modelling the prior $p(y)$


For a binary classification problem, recall that maximum likelihood estimation of $p(y=\ell)$ (the probability that we get an example belonging to class $\ell \in \{0,1\}$) is given by
$$p(y = \ell) = \frac{\sum_{i=1}^n 1[y^{(i)} = \ell]}{n}$$
where for a boolean predicate $q$, $1[q]$ is $1$ if $q$ is true and $0$ if $q$ is false. In other words, the estimate for $p(y = \ell)$ is the fraction of training examples with output $\ell$. 

Implement the function `priory`, that returns this estimate based on the training set. Since $p(y=0) = 1-p(y=1)$, you can recover all necessary information from $p(y=1)$. Your function should return a single value namely $p(y=1)$.  

**Remark**: Try to avoid loops.

In [5]:
#grade

def priory(train_labels):
    '''
    Parameters:
      train_labels: shape: (N,). N is the number of training examples.
    Return:
      log_py: a number.
    '''
    n = len(train_labels)
    py = np.sum(train_labels)/n

    ### START STUDENT CODE ##

    ### END CODE ###

    return py

In [6]:
# Sample Test Case
some_labels = np.array([0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1])
some_py = priory(some_labels)
assert (some_py == 0.6)

Let's see what is the `py` value for our dataset.

In [7]:
py = priory(target)
py

0.27

### 1.2.2 Task 2: Modelling $p(x^{(j)}|y)$

Recall that in Naive Bayes, each input feature is Boolean valued (i.e., either $0$ or $1$). Further the maximum likelihood estimate of $p(x_j=e | y = \ell)$ (probability that the $j$th feature takes value $e \in \{0,1\}$ given that the output is $\ell$) is 
$$p(x_j = e | y = \ell) = \frac{\sum_{i=1}^n 1[x^{(i)}_j = e \wedge y^{(i)} = \ell]}{\sum_{i=1}^n 1[y^{(i)} = \ell]}.$$
In other words, $p(x_j=e | y = \ell)$ is the fraction among all training examples that have output label $\ell$, those that in addition have their $j$th feature take value $e$.

We need to get all such conditional probabilities for each feature and each output value.

As mentioned in class, often the training set may contain no examples where particular feature takes on a given value $e$, and when this happens, the above estimates will return $0$. This can be undesirable in many cases. Instead, in practice, one uses Laplace smoothing, where the following estimates are used.

$$p(x_j = e | y = \ell) = \frac{1+\sum_{i=1}^n 1[x^{(i)}_j = e \wedge y^{(i)} = \ell]}{2+\sum_{i=1}^n 1[y^{(i)} = \ell]}.$$

Now, implement the `cond` function, which uses Laplace smoothing and returns a matrix like below:
$$\begin{bmatrix} p(x_0=1|y=0) &  p(x_0 = 1|y=1)\\
p(x_1=1|y=0) & p(x_1 = 1|y=1) \\
\cdots & \cdots \\
p(x_d=1|y=0) & p(x_d = 1|y=1)\end{bmatrix}$$
whose shape is `(d,2)`, where `d` is the number of features in the training set.

**Remark**: We don't compute $p(x_j=0|y)$ because it is equal to $1-p(x_j=1|y)$

In [83]:
#grade

def cond(train_features, train_labels):
    '''
    Parameters:
      train_features: X.shape: (N, d)
      train_labels: Y.shape: (N,)
    Return:
      px_y: shape: (d,2)
    '''
    N, d = train_features.shape
    px_y = np.empty((d,2))
    ### START STUDENT CODE ###
    
    denom1 = 2 + (N - np.sum(train_labels))
    denom2 = 2 + np.sum(train_labels)

    
    counts1 = np.zeros(d)
    counts2 = np.zeros(d)
    
    for i, features in enumerate(train_features):
        
        
    
        #print(features, train_labels[i])
        
        for j, feature in enumerate(features):
            
            if feature == 1 and train_labels[i] == 0:
                
                counts1[j] += 1
                
            elif feature == 1 and train_labels[i] == 1:
                
                counts2[j] += 1
 
    
    px_y[:,0] = (counts1 + 1) / denom1
    px_y[:,1] = (counts2 + 1) / denom2
            
            
        
    
    #print(np.sum(train_features, axis = 0))
    ### END CODE ###
    assert px_y.shape == (d, 2)
    return px_y

In [84]:
# Sample Test Case
some_feats = np.array([[0, 1, 1, 0, 0],
                       [1, 0, 1, 0, 1],
                       [1, 1, 0, 1, 0, ]])
some_labels = np.array([0, 1, 0])
some_px_y = cond(some_feats, some_labels)
assert np.array_equal(some_px_y.round(2), np.array([[0.5      ,  0.66666667],
                                                    [0.75     ,  0.33333333],
                                                    [0.5      ,  0.66666667],
                                                    [0.5      ,  0.33333333],
                                                    [0.25     ,  0.66666667]]).round(2))

Let's see what is the `px_y` value for our dataset.

In [85]:
px_y = cond(data, target)
px_y[:5]

array([[0.13333333, 0.20689655],
       [0.37333333, 0.79310345],
       [0.82666667, 0.5862069 ],
       [0.50666667, 0.31034483],
       [0.2       , 0.20689655]])

### 1.2.3 Task 3: Classify

We have now built the functions to calculate the prior $p(y)$ and $p(x_j|y)$. Using these functions, let us build a function classifies a new data point $x \in \{0,1\}^d$ using these priors. The function `classify` takes 
- `py`: An estimate for $p(y = 1)$
- `px_y`: A `(d,2)` matrix that contains estimates of $p(x_j=e | y = \ell)$
- `train_features`: A `(N,d)` matrix containing `N` new examples that need to be classified

and returns `(N,)` array that contains the classification for each example.

Recall that to classify a new data point $x = [x^{(1)}, ..., x^{(d)}]^\top$, we need to choose $y$ such that
$$\bigg[\log p(y) + \sum_{j=1}^d \log p(x^{(j)}|y) \bigg]$$ is the largest.

In [253]:
#grade

def classify(py, px_y, train_features):
    '''
    Parameters:
      py: a number
      px_y: shape: (d,2)
      train_features: shape: (N, d)
    Return:
      pred: shape: (N,)
    '''
    N, d = train_features.shape
    pred = []
    ### START STUDENT CODE ###
    
    for features in train_features:

        s0 = features@px_y[:,0]
        s1 = features@px_y[:,1]
        
        if s0 > s1:
            
            pred.append(0)
            
        else:
            
            pred.append(1)


            
    return np.array(pred)

    
    ### END CODE ###

In [254]:
# Sample Test Case
some_feats = np.array([[0, 1, 1, 0, 0],
                       [1, 0, 1, 0, 1],
                       [1, 1, 0, 1, 0]])

some_py = 1./3
some_cond = np.array([[1./2, 2./3],
                     [3./4, 1./3],
                     [1./2, 2./3],
                     [1./2, 1./3],
                     [1./4, 2./3]])

pred = classify(some_py, some_cond, some_feats)
print(pred)
ans = np.array([0, 1, 0])
assert np.array_equal(pred,ans)

[0 1 0]


Let's see what is the the training accuracy for our dataset.

In [234]:
pred = classify(py, px_y, data).squeeze()
print("Training Accuracy: {0:0.2f}%".format(100*(pred == target).mean()))

Training Accuracy: 85.00%


## 1.3 Test

Let's load the test data, and see the testing accuracy!

In [111]:
tdata, ttarget = bayes_dataset('test')

In [112]:
pred_test = classify(py, px_y, tdata).squeeze()
print("Testing Accuracy: {0:0.2f}%".format(100*(pred_test == ttarget).mean()))

Testing Accuracy: 94.00%


# 2. Gaussian Naive Bayes Classifier

Gaussian Naive Bayes Classifier is a special, simpler form of the Gaussian Discriminant Analysis (GDA) that was discussed in class. Unlike the Naive Bayes method where the input features are Boolean valued, in this approach input features taken on real values. The assumption is (as in Naive Bayes and GDA) $p(y=1)$ is assumed to be distributed according to a Bernouli distribution. In addition, given the output, the input features are assumed to be distributed accordining to the Gaussian distribution. Like Naive Bayes, we assume that the conditional distribution of each feature is independent of the other features --- this simplifying assumption allows one to model the distribution of each feature conditional on the output as a *uni-variate* Gaussian as opposed to a multi-variate distribution. It simplifies the maximum likelihood analysis, which is spelt out below.

## 2.1 Dataset
Now let's look at a real-world case. Let's try the diabetes dataset.

### 2.1.1 Description


The UC Irvine's Machine Learning Data Repository Department hosts a Kaggle Competition with famous collection of data on whether a patient has diabetes (the Pima Indians dataset), originally owned by the National Institute of Diabetes and Digestive and Kidney Diseases and donated by Vincent Sigillito. 

You can find this data at https://www.kaggle.com/uciml/pima-indians-diabetes-database/data. The Kaggle website offers valuable visualizations of the original data dimensions in its dashboard. It is quite insightful to take the time and make sense of the data using their dashboard before applying any method to the data.

**Information Summary**

* **Input/Output**: This data has a set of attributes of patients, and a categorical variable telling whether the patient is diabetic or not. 

* **Missing Data**: For several attributes in this data set, a value of 0 may indicate a missing value of the variable. But here we just ignore that these zero values are missing values.

* **Final Goal**: We want to build a classifier that can predict whether a patient has diabetes or not.

### 2.1.2 Loading Data

In [113]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [114]:
# Let's generate the split ourselves.
np_random = np.random.RandomState(seed=12345)
rand_unifs = np_random.uniform(0,1,size=df.shape[0])
division_thresh = np.percentile(rand_unifs, 80)
train_indicator = rand_unifs < division_thresh
eval_indicator = rand_unifs >= division_thresh

In [115]:
train_df = df[train_indicator].reset_index(drop=True)
train_features = train_df.loc[:, train_df.columns != 'Outcome'].values
train_labels = train_df['Outcome'].values
train_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [116]:
eval_df = df[eval_indicator].reset_index(drop=True)
eval_features = eval_df.loc[:, eval_df.columns != 'Outcome'].values
eval_labels = eval_df['Outcome'].values

In [117]:
train_features.shape, train_labels.shape, eval_features.shape, eval_labels.shape

((614, 8), (614,), (154, 8), (154,))

## 2.2 Train

The prior distributions are setup as follows:
$$
\begin{array}{c}
y \sim \mbox{Bernoulli}(\phi)\\
x_j | y = 0 \sim \mathcal{N}(\mu_0(j),\sigma_0(j))\\
x_j | y = 1 \sim \mathcal{N}(\mu_1(j),\sigma_1(j))
\end{array}
$$

We also make the Naive Bayes assumption that each feature value conditioned on the output is independent. Thus
$$p(x|y=\ell) = \prod_{j=1}^d p(x_j | y = \ell).$$

Given these assumptions, we will choose the parameters $\phi, \{\mu_0(j),\mu_1(j),\sigma_0(j),\sigma_1(j)\}_{j=1}^d$ such that the likelihood of the training set is maximized.

### 2.2.1 Estimating the prior $p(y)$

Maximum likelihood analysis, when carried out, determines that the estimate for $p(y=1)$ is the same as before, i.e.,$$p(y = 1) = \frac{\sum_{i=1}^n 1[y^{(i)} = 1]}{n}.$$

The function `priory` we have already implemented computes this quantity, and we can reuse it here.

In [118]:
# priory

### 2.2.2 Task 4: Estimating the Gaussian means

We will now estimate the means of the Gaussian distributions that determine the value of the features, i.e., the parameters $\{\mu_0(j),\mu_1(j)\}_{j=1}^d$. Again maximum likelihood estimation will suggest a natural estimate for these parameters, i.e., the mean value of a particular feature when restricted to the samples that have a particular output.
$$\mu_\ell(j) = \frac{\sum_{i=1}^n x^{(i)}_j 1[y^{(i)} = \ell]}{\sum_{i=1}^n 1[y^{(i)} = \ell]}$$

Write a function `get_mu_y` that takes the numpy arrays `train_features` and `train_labels` as input, and outputs a matrix `mu_y` of shape `(d,2)`, where `d` is the number of features.

Some points regarding this task:

* **You can assume that `train_features` has no missing elements in this task**.

* Try and avoid the utilization of loops as much as possible. No loops are necessary.

In [156]:
#grade

def get_mu_y(train_features, train_labels):
    '''
    Parameters:
      train_features: shape: (N,d). N is the number of training data points, and d is the number of the features. 
      train_labels: shape: (N,) 
    Return:
      mu_y: shape: (d,2)
    '''
    N, d = train_features.shape
    
    ### START STUDENT CODE ###
    
    county0 = N - np.sum(train_labels)
    county1 = np.sum(train_labels)
    
    col1 = np.zeros(d)
    col2 = np.zeros(d)
    
    for j in range(d):
        
        xjs = np.sum(train_features[:,j] * train_labels)
        
        
        #print(xjs/county1)
        col2[j] = xjs/county1
        
        xjs = np.sum(train_features[:,j] * abs(train_labels - 1))
        
        
        #print(xjs/county0)
        col1[j] = xjs/county0
    
    
    
    
    ### END CODE ###
    mu_y = np.empty((d,2))
    mu_y[:,0] = col1
    mu_y[:,1] = col2
    
    assert mu_y.shape == (d, 2)
    return mu_y

In [157]:
# Sample Test Case
some_feats = np.array([[  1. ,  85. ,  66. ,  29. ,   0. ,  26.6,   0.4,  31. ],
                       [  8. , 183. ,  64. ,   0. ,   0. ,  23.3,   0.7,  32. ],
                       [  1. ,  89. ,  66. ,  23. ,  94. ,  28.1,   0.2,  21. ],
                       [  0. , 137. ,  40. ,  35. , 168. ,  43.1,   2.3,  33. ],
                       [  5. , 116. ,  74. ,   0. ,   0. ,  25.6,   0.2,  30. ]])
some_labels = np.array([0, 1, 0, 1, 0])
some_mu_y = get_mu_y(some_feats, some_labels)
assert np.array_equal(some_mu_y.round(2), np.array([[  2.33,   4.  ],
                                                    [ 96.67, 160.  ],
                                                    [ 68.67,  52.  ],
                                                    [ 17.33,  17.5 ],
                                                    [ 31.33,  84.  ],
                                                    [ 26.77,  33.2 ],
                                                    [  0.27,   1.5 ],
                                                    [ 27.33,  32.5 ]]))

### 2.2.3 Task 5: Estimating the Gaussian standard deviations

We will now estimate the standard deviations of the Gaussian distributions that determine the value of the features, i.e., the parameters $\{\sigma_0(j),\sigma_1(j)\}_{j=1}^d$. Again maximum likelihood estimation will suggest a natural estimate for these parameters, i.e., the standard deviation of a particular feature when restricted to the samples that have a particular value.
$$(\sigma_\ell(j))^2 = \frac{\sum_{i=1}^n (x^{(i)}_j-\mu_\ell(j))^2 1[y^{(i)} = \ell]}{\sum_{i=1}^n 1[y^{(i)} = \ell]}.$$
In other words, $\sigma_\ell(j)$ is the standard deviation in the $j$th feature when restricted to examples that have output $\ell$.

Write a function `get_sigma_y` that takes the numpy arrays `train_features` and `train_labels` as input, and outputs the matrix `\sigma_y` with shape `(d,2)`, where `d` is the number of features. 

Some points regarding this task:

* **You can assume that `train_features` has no missing elements in this task**.

* Try and avoid the utilization of loops as much as possible. No loops are necessary.

In [162]:
#grade

def get_sigma_y(train_features, train_labels):
    '''
    Parameters:
      train_features: shape: (N,d). N is the number of training data points, and d is the number of the features. 
      train_labels: shape: (N,) 
    Return:
      sigma_y: shape: (d,2)
    '''
    N, d = train_features.shape
    
    ### BEGIN STUDENT CODE ###
    
    county0 = N - np.sum(train_labels)
    county1 = np.sum(train_labels)
    
    col1 = np.zeros(d)
    col2 = np.zeros(d)
    mu_y = get_mu_y(train_features, train_labels)

    
    for j in range(d):
        
        xjs = np.sum( (train_features[:,j] - mu_y[j][1])**2 * train_labels)
        

        col2[j] = np.sqrt(xjs/county1)
        
        xjs = np.sum( (train_features[:,j] - mu_y[j][0])**2 * abs(train_labels - 1))
        
        

        col1[j] = np.sqrt(xjs/county0)
    
    
    
    ### END CODE ###
    sigma_y = np.empty((d,2))
    sigma_y[:,0] = col1
    sigma_y[:,1] = col2
    
    assert sigma_y.shape == (d, 2)
    return sigma_y

In [164]:
# Sample Test Case
some_feats = np.array([[  1. ,  85. ,  66. ,  29. ,   0. ,  26.6,   0.4,  31. ],
                       [  8. , 183. ,  64. ,   0. ,   0. ,  23.3,   0.7,  32. ],
                       [  1. ,  89. ,  66. ,  23. ,  94. ,  28.1,   0.2,  21. ],
                       [  0. , 137. ,  40. ,  35. , 168. ,  43.1,   2.3,  33. ],
                       [  5. , 116. ,  74. ,   0. ,   0. ,  25.6,   0.2,  30. ]])
some_labels = np.array([0, 1, 0, 1, 0])
some_std_y = get_sigma_y(some_feats, some_labels)
assert np.array_equal(some_std_y.round(3), np.array([[ 1.886,  4.   ],
                                                     [13.768, 23.   ],
                                                     [ 3.771, 12.   ],
                                                     [12.499, 17.5  ],
                                                     [44.312, 84.   ],
                                                     [ 1.027,  9.9  ],
                                                     [ 0.094,  0.8  ],
                                                     [ 4.497,  0.5  ]]))

## 2.3 Task 6: Classification

Having computed the priors, we can use the estimate to classify a new example $x$. Recall that the MAP rule (which we also used in Naive Bayes and logistic regression), suggests that, on input $x$, we choose the output $\ell \in \{0,1\}$ that maximizes the probability $p(y = \ell | x)$. The conditional $p(y = \ell | x)$ can be computed using Bayes rule using the priors, and maximizing $p(y = \ell | x)$ is equivalent to maximizing $p(x, y = \ell)$, which can be written as 
$$\prod_{j=1}^d p(x_j | y = \ell)\cdot p(y = \ell).$$
Again we can instead maximize the log of this expression, which can be written as
$$\log p(y = \ell) + \sum_{j=1}^d \log p(x_j | y = \ell).$$
Finally, recall that the PDF of the univariate Gaussian distribution is
$$ p(x_j | y = \ell) = \frac{1}{\sqrt{2\pi(\sigma_\ell(j))^2}} \exp (\frac{-(x_j-\mu_\ell(j))^2}{2(\sigma_\ell(j))^2}).$$

Write a function `gaussian_classify` that takes the numpy arrays `train_features` (new examples to be classified), `mu_y` (estimates of the Gaussian means), `sigma_y` (estimates of Gaussian standard deviations), and  `p_y` (estimate of $p(y = 1)$) as input, then outputs the matrix `log_p_x_y` with the shape `(N, 2)`, where `N` is the number of new examples. The matrix `log_p_x_y` has the form

$$\log p_{x,y} = \begin{bmatrix} \bigg[\log p(y=0) + \sum_{j=0}^{d} \log p(x_j^{(1)}|y=0) \bigg] & \bigg[\log p(y=1) + \sum_{j=0}^{d} \log p(x_j^{(1)}|y=1) \bigg] \\
\bigg[\log p(y=0) + \sum_{j=0}^{d} \log p(x_j^{(2)}|y=0) \bigg] & \bigg[\log p(y=1) + \sum_{j=0}^{d} \log p(x_j^{(2)}|y=1) \bigg] \\
\cdots & \cdots \\
\bigg[\log p(y=0) + \sum_{j=0}^{d} \log p(x_j^{(N)}|y=0) \bigg] & \bigg[\log p(y=1) + \sum_{j=0}^{d} \log p(x_j^{(N)}|y=1) \bigg] \\
\end{bmatrix}$$

where $x^{(i)}$ is the $i$th new example in `train_features`.

Try and avoid the utilization of loops as much as possible. No loops are necessary.

**Hint**: You may find it useful to take the log of the Gaussian PDF on paper before implementing the function. 

**Important Note**: Do not use third-party and non-standard implementations for computing $\log p(x_i^{(j)}|y)$. Using functions that find the Gaussian PDF, and then taking their log is **numerically unstable**; the Gaussian PDF values can easily become extremely small numbers that cannot be represented using floating point standards and thus would be stored as zero. Taking the log of a zero value will throw an error. On the other hand, it is unnecessary to compute and store $p(x_i^{(j)}|y)$ in order to find $\log p(x_i^{(j)}|y)$; **you can write $\log p(x_i^{(j)}|y)$ as a direct function of `mu_y`, `sigma_y` and the features. This latter approach is numerically stable, and can be applied when the PDF values are much smaller than could be stored using the common standards.**

**Remark**: Print the shape of each matrix when you feel confused.

**Remark**: When `axis=1`, certain functions remove the second number from the matrix's `shape`

In [209]:
#grade

def gaussian_classify(train_features, mu_y, sigma_y, py):
    '''
    Parameters:
      train_features: shape: (N, d)
      mu_y: shape: (d, 2)
      sigma_y: shape: (d, 2)
      py: number
    Return:
      log_p_x_y: shape: (N, 2). the above matrix
      pred: shape: (N, 1). The prediction results
    '''
    N, d = train_features.shape
    
    ### BEGIN STUDENT CODE ###
    
    log_p_x_y = np.empty((N,2))
    s0 = 0
    s1 = 0
    
    for j in range(d):
        
        xjs = train_features[:,j]
        
        z = .5 * (xjs - mu_y[j][0])**2 / sigma_y[j][0]**2
        
        c = .5 * np.log(2*np.pi)
        s0 += -np.log(sigma_y[j][0]) - c - z
        
        z = .5 * (xjs - mu_y[j][1])**2 / sigma_y[j][1]**2
        
        c = .5 * np.log(2*np.pi)
        s1 += -np.log(sigma_y[j][1]) - c - z
        
    s0 += np.log(1-py)
    s1 += np.log(py)
    
    log_p_x_y[:,0] = s0
    
    log_p_x_y[:,1] = s1
    
    pred = []
    
    for row in log_p_x_y:

        s0 = row[0]
        s1 = row[1]      
        
        if s0 > s1:
            
            pred.append(0)
            
        else:
            
            pred.append(1)
            
    pred = np.array(pred)
        
        
    

    ### END CODE ###
    
    assert log_p_x_y.shape == (N,2)
    assert pred.shape == (N,)
    return log_p_x_y, pred

In [204]:
# Sample Test Case
some_feats = np.array([[  1. ,  85. ,  66. ,  29. ,   0. ,  26.6,   0.4,  31. ],
                       [  8. , 183. ,  64. ,   0. ,   0. ,  23.3,   0.7,  32. ],
                       [  1. ,  89. ,  66. ,  23. ,  94. ,  28.1,   0.2,  21. ],
                       [  0. , 137. ,  40. ,  35. , 168. ,  43.1,   2.3,  33. ],
                       [  5. , 116. ,  74. ,   0. ,   0. ,  25.6,   0.2,  30. ]])
some_labels = np.array([0, 1, 0, 1, 0])
some_py = 0.4
some_mu_y = np.array([[  2.33333333,   4.        ],
       [ 96.66666667, 160.        ],
       [ 68.66666667,  52.        ],
       [ 17.33333333,  17.5       ],
       [ 31.33333333,  84.        ],
       [ 26.76666667,  33.2       ],
       [  0.26666667,   1.5       ],
       [ 27.33333333,  32.5       ]])
some_std_y = np.array([[ 1.88561808,  4.        ],
       [13.76791762, 23.        ],
       [ 3.77123617, 12.        ],
       [12.49888884, 17.5       ],
       [44.31202495, 84.        ],
       [ 1.02740233,  9.9       ],
       [ 0.0942809 ,  0.8       ],
       [ 4.49691252,  0.5       ]])
some_log_p_x_y, some_pred = gaussian_classify(some_feats, some_mu_y, some_std_y, some_py)
assert np.array_equal(some_log_p_x_y.round(2), np.array([[ -20.822,  -36.606],
                                                         [ -60.879,  -27.944],
                                                         [ -21.774, -295.68 ],
                                                         [-417.359,  -27.944],
                                                         [ -23.2  ,  -42.6  ]]).round(2))
assert np.array_equal(some_pred, some_labels)

## 2.4 Test

### 2.4.1 Integrate all the functions together

In [205]:
class NBClassifier():
    def __init__(self, train_features, train_labels):
        self.train_features = train_features
        self.train_labels = train_labels
        self.py = priory(train_labels)
        self.mu_y = self.get_cc_means()
        self.sigma_y = self.get_cc_std()
        
    def get_cc_means(self):
        mu_y = get_mu_y(self.train_features, self.train_labels)
        return mu_y
    
    def get_cc_std(self):
        sigma_y = get_sigma_y(self.train_features, self.train_labels)
        return sigma_y
    
    def predict(self, features):
        _, pred = gaussian_classify(features, self.mu_y, self.sigma_y, self.py)
        return pred

### 2.4.2 Running our Classifier on Diabetes Dataset

In [206]:
diabetes_classifier = NBClassifier(train_features, train_labels)
train_pred = diabetes_classifier.predict(train_features)
eval_pred = diabetes_classifier.predict(eval_features)

In [207]:
train_acc = (train_pred==train_labels).mean()
eval_acc = (eval_pred==eval_labels).mean()
print(f'The training data accuracy of your trained model is {train_acc}')
print(f'The evaluation data accuracy of your trained model is {eval_acc}')

The training data accuracy of your trained model is 0.7671009771986971
The evaluation data accuracy of your trained model is 0.7532467532467533


### 2.4.3 Running an off-the-shelf implementation of Gaussian Naive-Bayes For Comparison

In [208]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(train_features, train_labels)
train_pred_sk = gnb.predict(train_features)
eval_pred_sk = gnb.predict(eval_features)
print(f'The training data accuracy of your trained model is {(train_pred_sk == train_labels).mean()}')
print(f'The evaluation data accuracy of your trained model is {(eval_pred_sk == eval_labels).mean()}')

The training data accuracy of your trained model is 0.7671009771986971
The evaluation data accuracy of your trained model is 0.7532467532467533
